Meanshift (Ortalama Kayma), noktaları moda doğru kaydırarak veri noktalarını kümelere yinelemeli olarak atayan bir kümeleme algoritmasıdır.
Mod, en yüksek veri noktası yoğunluğu olarak tanımlanabilir.
Bu nedenle, mod arama algoritması olarak da bilinir.

In [ ]:
#kutuphaneleri import edelim
import cv2
import numpy as np 

#kamerayi ac
cap = cv2.VideoCapture(0)

#bir frame oku
ret, frame = cap.read()

#kamera acildi mi
if ret == False:
    print("uyarı")
    
#eğitilmiş haarcascade içeri alalım
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#detectMultiScale sonucunda bize bir rectangle iletiliyor
face_rects= face_cascade.detectMultiScale(frame)

# yüzün koordinatlarını ve boyutlarını belirlemek için
(face_x, face_y,w,h)=tuple(face_rects[0])
track_window=(face_x, face_y,w,h) #meanshift algoritması için girdi hazırladım

#region of interest #tespit ettiğim kutucuğun içi
roi = frame[face_y:face_y+h, face_x: face_x+w] #roi = yüz

#hsv formatına ceviriyoruz
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

#hsv_roi histogramini hesaplayacağim
# [img], maskeleme, [180] histogram 0 ile 179 arasındaki renk tonlarını temsil eder, range et yani sırayla say  
roi_hist = cv2.calcHist([hsv_roi],[0], None,[180],[0,180]) #takip için histogram gerekli

#roilerimi 0 ile 255 arasında sıkıştırıyorum
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

#takip algoritması için gerekli durdurma kriterleri belirleyelim 
#caunt = hesaplanacak max oge sayisi
#eps= istenen doğruluk veya değişiklik diyebiliriz
#5 yineleme veya 1 tane epsilon
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5,1)


while True:
    ret,frame=cap.read()
    
    if ret:
        
        #kameradan aldigim goruntuyu hsv formatina ceviriyorum
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        
        #histogramı bir goruntude bulmak için kullanıyoruz
        # goruntu, 0 dan, hist kadar, 0-180 arasındaki renkler, 1: projeksiyon sonucu 0 ile 255 arasında normalleştirilir.  
        dst = cv2.calcBackProject([hsv],[0],roi_hist, [0,180],1)#piksel karşılaştırma
        
        #track_window: takip edilecek nesnenin başlangıç pencere bölgesini temsil eder.
        #term_crit: Bu, sonlanma kriterleri
        ret, track_window = cv2.meanShift(dst,track_window, term_crit)
        
        #track_window : takip ettigi icin nesnenin yeni konumunu döndürecek
        x,y,w,h = track_window
        
        #yeni bir resim oluşturup dikdortgen cizecegim
        #resim, bas konum , son konum, renk, kalınlık
        img2 = cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),5)
        
        
        cv2.imshow("takip", img2)
        
        #q basinca kapat
        if cv2.waitKey(0) & 0xFF == ord("q"): break
cap.relase()
cv2.destroyAllWindows()
